In [82]:
import pandas as pd

In [83]:
def split_years(df):
    df['year_4'] = df['resources.description'].str.extract(r'(\d{4})')
    df['resources.description'] = df['resources.description'].str.replace(r'\d{4}', '', regex=True)
    df['year_2'] = df['resources.description'].str.extract(r'(\d{2})')
    df['resources.description'] = df['resources.description'].str.replace(r'\d{2}', '', regex=True)
    df['year_2'] = pd.to_datetime(df['year_2'], format='%y').dt.strftime('%Y')
    df['year_2'] = df['year_2'].fillna(0).astype(int)
    df['year_4'] = df['year_4'].fillna(0).astype(int)
    df['year'] = df['year_4'] + df['year_2']
    df = df.drop(['year_4', 'year_2'], axis=1)
    print(df.dtypes)
    return df


In [84]:
def clean_Subscriptions_data(subset):
    subset['resources.description'] = subset['resources.description'].str.replace(')', '')
    subset['resources.description'] = subset['resources.description'].str.replace('(', '')
    subset['resources.description'] = subset['resources.description'].str.replace('  ', ' ')
    subset['resources.description'] = subset['resources.description'].str.split(':', expand=False)
    subset['schedule'] = schedule
    subset['section_month'] = subset['resources.description'].str[1]
    subset['section_month'] = subset['section_month'].str.split(' ', expand=False)
    subset['section_month'] = subset['section_month'].apply(lambda x: [item for item in x if item])
    subset['section'] = subset['section_month'].str[0]
    subset['month'] = subset['section_month'].str[1]
    subset = subset.drop(['section_month'], axis=1)
    
    return subset


In [85]:
def clean_data(file_path):
    df = pd.read_csv(file_path)

    columns_to_keep = ['resources.description', 'gross_amount', 'gocardless_fees', 'app_fees', 'net_amount', 'payouts.arrival_date', 'payments.metadata.Member', 'payments.metadata.References']
    df = df[columns_to_keep]
    df = split_years(df)

    payment_schedules = ['Subscriptions', 'Activities', ]

    dfs = []

    for schedule in payment_schedules:
        subset = df[df['resources.description'].str.contains(schedule)].copy()
        if schedule == 'Subscriptions':
            subset = clean_Subscriptions_data(subset)
        

        dfs.append(subset)

    df = pd.concat(dfs, ignore_index=True)







    return df

df = clean_data(file_path='inputs_3/payout_transactions_reconciliation-export-EX00036AZABPPE.csv')
df.sample(5)


resources.description            object
gross_amount                    float64
gocardless_fees                 float64
app_fees                        float64
net_amount                      float64
payouts.arrival_date             object
payments.metadata.Member         object
payments.metadata.References     object
year                              int64
dtype: object


,resources.description,gross_amount,gocardless_fees,app_fees,net_amount,payouts.arrival_date,payments.metadata.Member,payments.metadata.References,year,schedule,section,month
9,"[Monthly Subscriptions, Scouts January]",12.0,0.29,0.23,11.48,2024-02-12 00:00:00,Joshua Lowe (1279472),714273 - 57594 - 19515,2024,Subscriptions,Scouts,January
46,"[Monthly Subscriptions, Beavers January ]",12.0,0.29,0.23,11.48,2024-02-12 00:00:00,Chloe Jenkins (2535170),743315 - 65921 - 16992,2024,Subscriptions,Beavers,January
127,"Activities (Camps, Sleepovers etc): Scouts (Su...",30.0,0.46,0.59,28.95,2024-02-12 00:00:00,Hudson Danks (2504050),741851 - 57593 - 19515,0,NaN,NaN,NaN
138,"Activities (Camps, Sleepovers etc): Scouts ( L...",58.0,0.71,1.13,56.16,2024-02-12 00:00:00,Hudson Danks (2504050),730026 - 57593 - 19515,2024,NaN,NaN,NaN
63,"[Monthly Subscriptions, Squirrels January]",12.0,0.29,0.23,11.48,2024-02-12 00:00:00,Jude Tunstall (2734925),716136 - 67369 - 69829,2024,Subscriptions,Squirrels,January
